In [1]:
import sys
sys.path.append('../')

In [2]:
from scripts.utils import data_path, set_mpl
from scripts.cross_match_scripts import  assess_goodnes_of_cross_match, rayleigh_plot
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.display import display
from astropy.coordinates import SkyCoord
from astropy import units as u
set_mpl()
%matplotlib inline

matplotlib settings set


matplotlib settings set
matplotlib settings set


In [14]:
nway_all = pd.read_pickle(data_path+'ERO_lhpv_03_23_sd01_a15_g14_desi_nway_match_all_candidates.gz_pkl', compression='gzip')
nway_best = nway_all.query('nway_match_flag == 1')


hostless_cols = ['srcname_fin', 'RA_fin', 'DEC_fin', 'nway_Separation_EROSITA_DESI', 'pos_r98', 'nway_prob_has_match', 'flux_05-20', 'ML_CTS_0', 'DET_LIKE_0', 'nway_photometry_nnmag_grzw1w2_orig', 'nway_photometry_nnmag_grzw1_orig', 'nway_photometry_nnmag_grz_orig']


In [4]:
nway_all = nway_all.query('DET_LIKE_0>10')
nway_best = nway_best.query('DET_LIKE_0>10')

## How many hostless candidates with high confidence?

In [5]:
query = " DET_LIKE_0>20 & nway_prob_has_match < 0.05 "
nway_hostless = nway_best.query(query)
nway_not_hostless = nway_best.query("DET_LIKE_0>20 & nway_prob_has_match > 0.7")
nway_hostless['outside_r98'] = nway_hostless['nway_Separation_EROSITA_DESI'] > nway_hostless['pos_r98']
nway_hostless.outside_r98.value_counts()

True     130
False     14
Name: outside_r98, dtype: int64

In [6]:
from astroquery.simbad import Simbad
customSimbad = Simbad()
# https://github.com/astropy/astroquery/blob/main/astroquery/simbad/data/votable_fields_dict.json
customSimbad.add_votable_fields(
    'distance_result', 'id(1)',  'otype', 'z_value',  'plx', 'pm')

def query_simbad(ra, dec, radius):
    coords = SkyCoord(ra, dec, unit='deg')
    simbad_table = customSimbad.query_region(coords, radius=radius * u.arcsec)
    if simbad_table is None:
        return None
    else:
        simbad_table = simbad_table.to_pandas()
        simbad_table = simbad_table.drop(['RA_PREC',	'DEC_PREC',	'COO_ERR_MAJA',	'COO_ERR_MINA',	'COO_ERR_ANGLE',	'COO_QUAL',	'COO_WAVELENGTH', 'COO_BIBCODE'], axis = 1)
        return simbad_table

In [ ]:
simbad_results = pd.DataFrame()
for i in range(len(nway_hostless)):
    ra = nway_hostless.iloc[i]['RA']
    dec = nway_hostless.iloc[i]['DEC']
    err = nway_hostless.iloc[i]['pos_r98']
    srcname = nway_hostless.iloc[i]['srcname_fin']

    simbad_table = query_simbad(ra, dec, 1.5*err)
    if simbad_table is not None:
        simbad_table['srcname_fin'] = srcname
        simbad_results = simbad_results.append(simbad_table)
simbad_results.set_index('srcname_fin', inplace=True)


In [8]:
nway_hostless.query("srcname_fin not in @simbad_results.index")
nway_hostless['simbad_found'] = nway_hostless['srcname_fin'].isin(simbad_results.index)

In [21]:
nway_hostless.query('simbad_found == False').sort_values(['ML_CTS_0', 'nway_prob_has_match'], ascending=[False, True])[hostless_cols]
#.to_csv(data_path+'hostless_candidates.csv', index=False)s

,srcname_fin,RA_fin,DEC_fin,nway_Separation_EROSITA_DESI,pos_r98,nway_prob_has_match,flux_05-20,ML_CTS_0,DET_LIKE_0,nway_photometry_nnmag_grzw1w2_orig,nway_photometry_nnmag_grzw1_orig,nway_photometry_nnmag_grz_orig
87513,SRGe J105429.0+575203,163.620843,57.867397,7.450001,2.386794,9.103829e-15,1.107308e-13,676.836548,1758.452148,0.854885,0.744525,0.071941
349,SRGe J102006.9+564541,155.028590,56.761414,6.374121,4.064868,1.256456e-02,1.478870e-13,357.072906,527.139282,0.916800,0.941837,0.680193
20843,SRGe J103410.4+580344,158.543238,58.062267,4.867712,2.897772,1.733940e-03,4.142073e-14,252.615128,441.016632,0.802147,0.896415,0.923010
78601,SRGe J105105.9+591618,162.774790,59.271725,4.517527,2.984301,3.214511e-04,3.993720e-14,242.460037,368.649536,-99.000000,-99.000000,-99.000000
16959,SRGe J103241.6+564652,158.173365,56.781007,6.036835,3.157361,2.086257e-06,3.634394e-14,223.694183,272.222076,-99.000000,-99.000000,0.063569
...,...,...,...,...,...,...,...,...,...,...,...,...
24978,SRGe J103540.8+604311,158.919860,60.719716,15.137637,10.217625,1.556736e-02,3.388274e-14,27.096478,24.421307,0.948420,0.955802,0.736293
98840,SRGe J110156.0+581025,165.483380,58.173590,7.598708,7.244496,4.125804e-02,7.373231e-15,26.360369,23.650047,0.542587,0.158212,0.170288
5978,SRGe J102657.4+555650,156.739093,55.947122,6.182407,6.507178,1.941515e-02,6.674068e-15,26.302002,22.257601,-99.000000,-99.000000,-99.000000
3461,SRGe J102418.2+570506,156.075874,57.084936,7.201363,7.435811,7.370018e-03,9.880049e-15,25.754313,23.867329,-99.000000,-99.000000,-99.000000


In [22]:
nway_not_hostless[hostless_cols]#.to_csv(data_path+'not_hostless_candidates.csv', index=False)

,srcname_fin,RA_fin,DEC_fin,nway_Separation_EROSITA_DESI,pos_r98,nway_prob_has_match,flux_05-20,ML_CTS_0,DET_LIKE_0,nway_photometry_nnmag_grzw1w2_orig,nway_photometry_nnmag_grzw1_orig,nway_photometry_nnmag_grz_orig
33,SRGe J101919.2+562429,154.830036,56.408006,0.650717,4.879086,0.995156,2.764135e-14,95.826859,85.512062,0.942408,0.942695,0.766463
91,SRGe J101925.5+561726,154.856092,56.290625,2.023571,4.372862,0.951605,3.790664e-14,138.071991,141.096008,-99.000000,0.799156,0.269563
151,SRGe J101939.4+562051,154.914334,56.347364,3.430252,6.840425,0.976747,1.141515e-14,47.077045,27.164392,0.950099,0.949118,0.584643
279,SRGe J102004.4+562331,155.018138,56.392082,1.982803,5.665856,0.983317,2.559419e-14,124.887436,148.769760,0.926579,0.906245,0.800109
312,SRGe J102005.6+562216,155.023477,56.371239,2.381510,4.239077,0.873980,1.921897e-14,95.014915,97.224068,0.680859,0.796471,0.513449
...,...,...,...,...,...,...,...,...,...,...,...,...
101094,SRGe J110607.9+582608,166.532952,58.435685,1.319125,2.472613,0.996475,6.099720e-13,631.891113,2028.946045,0.949998,0.959333,0.745629
101129,SRGe J110612.0+585212,166.550064,58.869999,1.859036,3.950482,0.992983,3.952924e-14,79.237862,159.248337,0.948984,0.946687,0.847364
101152,SRGe J110617.0+590146,166.570956,59.029519,1.287881,3.456700,0.996090,1.002464e-13,185.357910,456.035339,0.949944,0.962792,0.970178
101165,SRGe J110628.5+590311,166.618955,59.053068,5.580483,7.034747,0.747280,1.591821e-14,25.546541,22.433899,0.885835,0.942170,0.813107
